In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

### Get the list of tables in the dataset

In [ ]:
dataset_id = 'bigquery-public-data.ghcn_d'
tables = client.list_tables(dataset_id)  # Make an API request.

tables_in_noaa_dataset = []
for table in tables:
    tables_in_noaa_dataset.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
## We have data from 1763 to 2022
print("\n".join(tables_in_noaa_dataset[0:7]))
print('...')
print("\n".join(tables_in_noaa_dataset[(len(tables_in_noaa_dataset)-7):(len(tables_in_noaa_dataset))]))

### Sample a table

Include where qflag (quality flag) is null.

In [ ]:
_2021_data = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
LIMIT 10
"""
_2021_data_query_job = client.query(_2021_data)  # Make an API request.

_2021_data_query_job.to_dataframe()

## Find the stations within lat/long bounds

There are two ways to find this:
Manually establish bounds, i.e. by looking on Google Maps, for example:

burlington: 44.47   73.15 (per https://www.w3.org/2003/01/geo/test/ustowns/latlong.htm)
```
  latitude > 44.3
  AND latitude < 44.7
  AND longitude > -73.4
  AND longitude < -72.9
```
chicago:
```
  latitude > 41.7
  AND latitude < 42
  AND longitude > -87.7
  AND longitude < -87.5
```

Or use the legacy syntax to find those closest to central coordinates as described here: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/ghcn/ghcn_on_bq.ipynb

In [ ]:
burlington_query = """
SELECT
  id,
  name,
  state,
  latitude,
  longitude
FROM
  `bigquery-public-data.ghcn_d.ghcnd_stations`
WHERE
  latitude > 44.3
  AND latitude < 44.7
  AND longitude > -73.4
  AND longitude < -72.9
"""
burlington_query_job = client.query(burlington_query)  # Make an API request.

burlington_stations = burlington_query_job.to_dataframe()

In [ ]:
lat=44.47
lon=-73.15

close_to_query = f"""
SELECT
  name, id,
  state,
  latitude,
  longitude,
  DEGREES(ACOS(SIN(RADIANS(latitude)) * SIN(RADIANS({lat})) + COS(RADIANS(latitude)) * COS(RADIANS({lat})) * COS(RADIANS(longitude - {lon})))) * 60 * 1.515 * 1.609344 AS dist_kms
FROM
  [bigquery-public-data:ghcn_d.ghcnd_stations]
ORDER BY
  dist_kms ASC
LIMIT
  1000"""


In [ ]:
# Set use_legacy_sql to True to use legacy SQL syntax (this is necessary to use the DEGREES function)
job_config = bigquery.QueryJobConfig(use_legacy_sql=True)

burlington_query_close_to = client.query(close_to_query, job_config=job_config)  # Make an API request.

burlington_stations_close_to = burlington_query_close_to.to_dataframe()

In [ ]:
## list those stations within 30 kms
burlington_stations_close_to[burlington_stations_close_to.dist_kms <= 30]

## Sample recent weather data from the stations near Burlington, VT

In [ ]:
## Build query
stations_within_30_kms = list(burlington_stations_close_to[burlington_stations_close_to.dist_kms <= 30]['id'])

burlington_2022_sample = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2022`
WHERE qflag IS NULL
AND id IN ('{}')
LIMIT 10
""".format('\', \''.join(stations_within_30_kms))

In [ ]:
burlington_2022_sample

In [ ]:
burlington_2022_sample = client.query(burlington_2022_sample)  # Make an API request.

burlington_2022_sample_df = burlington_2022_sample.to_dataframe()

In [ ]:
burlington_2022_sample_df

In [ ]:
burlington_2021_time_series = """SELECT
*
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
AND id IN ('{}')
""".format('\', \''.join(stations_within_30_kms))

burlington_2021_data = client.query(burlington_2021_time_series)

burlington_2021_data_time_series = burlington_2021_data.to_dataframe()

In [ ]:
burlington_2021_data_time_series['element'] == 'SNOW'

In [ ]:
## Visualize
burlington_2021_data_time_series['date'] = pd.to_datetime(burlington_2021_data_time_series['date'])

burlington_snow_2021 = burlington_2021_data_time_series[burlington_2021_data_time_series['element'] == 'SNOW']

fig = px.scatter(burlington_snow_2021,x="date", y="value", color="id", title='Burlington VT Snow')
fig.show()

In [ ]:
fig = px.scatter(burlington_snow_2021.groupby(['date']).median().reset_index(),
                 x="date", y="value", title='Burlington VT Median Snow')
fig.show()

# Create a model

https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-glm

https://cloud.google.com/bigquery-ml/docs/arima-multiple-time-series-forecasting-tutorial

Example Time Series (ARIMA):
```
CREATE MODEL `project_id.mydataset.mymodel`
 OPTIONS(MODEL_TYPE='ARIMA_PLUS',
         time_series_timestamp_col='date',
         time_series_data_col='transaction') AS
SELECT
  date,
  transaction
FROM
  `mydataset.mytable`
```

Example linear regression (https://cloud.google.com/bigquery-ml/docs/linear-regression-tutorial):
```
#standardSQL
'''
CREATE OR REPLACE MODEL `bqml_tutorial.penguins_model`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['body_mass_g']) AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL
'''
```

#### About  models:
For Time Series (https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series)
* time_series_timestamp_col is what you'd expect
* time_series_data_col is what you're trying to predict

For Linear Regression:

* `input_label_cols` is the thing we're trying to predict
* The items in the `SELECT` statement are what's used to predict the input label

In [ ]:
stations_within_30km_text = '\', \''.join(stations_within_30_kms)
# standardSQL
snow_in_burlington_vt_arima = """
CREATE OR REPLACE MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`
 OPTIONS(MODEL_TYPE='ARIMA_PLUS',
         time_series_timestamp_col='date',
         time_series_data_col='mean_value') AS
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
UNION ALL
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2020`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
UNION ALL
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2019`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
""".format(stations_within_30km_text, stations_within_30km_text,
          stations_within_30km_text)


In [ ]:
snow_in_burlington_vt_arima

In [ ]:
burlington_2019_to_2021_model = client.query(snow_in_burlington_vt_arima)  # Make an API request.

## Get the training info

For Arima, it's somewhat limited:
https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train


In [ ]:
burlington_arima_training_info = """
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`)
"""

In [ ]:
burlington_2019_to_2021_model_training = client.query(burlington_arima_training_info)  # Make an API request.
burlington_2019_to_2021_model_training.to_dataframe()

### Get the coefficients

In [ ]:
burlington_arima_coefficents_query = """
SELECT
 *
FROM
 ML.ARIMA_COEFFICIENTS(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`)
 """

In [ ]:
burlington_arima_coefficents = client.query(burlington_arima_coefficents_query) 
burlington_arima_coefficents.to_dataframe()

## Evaluate the Model

ARIMA has a special function: https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-arima-evaluate


In [ ]:
burlington_arima_eval = """SELECT
  *
FROM
  ML.ARIMA_EVALUATE(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`, 
  STRUCT(FALSE AS show_all_candidate_models))"""

In [ ]:
burlington_2019_to_2021_model_eval = client.query(burlington_arima_eval)  # Make an API request.
burlington_2019_to_2021_model_eval.to_dataframe()

# Build a Forecast

https://cloud.google.com/bigquery-ml/docs/arima-single-time-series-forecasting-tutorial

This is 30 days into the future

```
SELECT
 *
FROM
 ML.FORECAST(MODEL bqml_tutorial.ga_arima_model,
             STRUCT(30 AS horizon, 0.8 AS confidence_level))
```

In [ ]:
burlington_arima_forecast_query = """SELECT
 *
FROM
 ML.FORECAST(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`,
             STRUCT(365 AS horizon, 0.8 AS confidence_level))"""

In [ ]:
burlington_arima_forecast_365 = client.query(burlington_arima_forecast_query)  # Make an API request.
burlington_arima_forecast_365_df = burlington_arima_forecast_365.to_dataframe()

In [ ]:
burlington_arima_forecast_365_df

In [ ]:
burlington_2022_time_series_query = """SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2022`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
""".format(stations_within_30km_text)

burlington_2022_time_series_data = client.query(burlington_2022_time_series_query)

burlington_2022_time_series_data_df = burlington_2022_time_series_data.to_dataframe()

In [ ]:
fig = px.scatter(burlington_2022_time_series_data_df,x="date", y="mean_value", title='Burlington VT Snow 2022')

fig.add_trace(go.Scatter(x=burlington_arima_forecast_365_df["forecast_timestamp"], 
                         y=burlington_arima_forecast_365_df["forecast_value"], 
                         mode="lines", name='Predicted Value'))
fig.show()